In [35]:
import yaml
import os
from pprint import pprint
from SmartAITool.core import bprint

# Load API configuration from a YAML file
with open("/home/rteam2/m15kh/LLM/config.yaml", "r") as file:
    config = yaml.safe_load(file)

os.environ["LANGSMITH_TRACING"] = config.get("LANGSMITH_TRACING", "false")
os.environ["LANGSMITH_API_KEY"] = config["LANGSMITH_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ['HUGGINGFACEHUB_API_TOKEN'] = config['HUGGINGFACEHUB_API_TOKEN']
GPT_API_KEY = config['GPT_API_KEY']


In [36]:
import re

def parse_srt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    blocks = re.split(r'\n\s*\n', content.strip())
    parsed_blocks = []

    for block in blocks:
        lines = block.strip().split('\n')
        if len(lines) >= 3:
            time_line = lines[1]
            time_match = re.match(r'(\d{2}:\d{2}:\d{2},\d{3})\s-->\s(\d{2}:\d{2}:\d{2},\d{3})', time_line)
            if time_match:
                start = time_match.group(1).replace(',', '.')
                end = time_match.group(2).replace(',', '.')
                text = " ".join(lines[2:])
                parsed_blocks.append({
                    "text": text,
                    "start_time": start,
                    "end_time": end
                })

    return parsed_blocks

# 📥 اجرای پارس روی فایل شما:
parsed_blocks = parse_srt("/home/rteam2/m15kh/LLM/srt/dan_persian.srt")

# تست:
pprint(parsed_blocks[:2])


[{'end_time': '00:00:05.080',
  'start_time': '00:00:00.790',
  'text': '[موسیقی] من که الان هوما شده همیشه میگم ای کاش یه'},
 {'end_time': '00:00:10.200',
  'start_time': '00:00:05.080',
  'text': 'کار کوچولوتری رو شروع کرده بودم اول کار پدرم درراومد شب\u200cهایی '
          'بوده که من گریه'}]


In [37]:
from langchain_core.documents import Document

docs = []
for block in parsed_blocks:
    docs.append(Document(
        page_content=block["text"],
        metadata={
            "start_time": block["start_time"],
            "end_time": block["end_time"]
        }
    ))

# تست:
print(docs[3].page_content)
print(docs[3].metadata)


نداریم وایستیم می‌ارزه این همه تلاش اینم کوشش می‌رزه تو اول انقدردی بازی می‌کنی و
{'start_time': '00:00:16.240', 'end_time': '00:00:21.800'}


In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# تعیین حداکثر طول chunk (مثلاً 2000 کاراکتر)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    separators=["\n", ".", "؟", "!", " "],
)

# تبدیل هر Document به subchunks
split_docs = text_splitter.split_documents(docs)

# نمایش تعداد و تست چند نمونه
print(f"تعداد تکه‌ها: {len(split_docs)}")
print(split_docs[0].page_content)
print(split_docs[0].metadata)


تعداد تکه‌ها: 965
[موسیقی] من که الان هوما شده همیشه میگم ای کاش یه
{'start_time': '00:00:00.790', 'end_time': '00:00:05.080'}


In [39]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

for i, split in enumerate(split_docs):
    num_tokens = len(encoding.encode(split.page_content))
    print(f"Split {i}: {num_tokens} tokens")


Split 0: 36 tokens
Split 1: 53 tokens
Split 2: 58 tokens
Split 3: 58 tokens
Split 4: 53 tokens
Split 5: 52 tokens
Split 6: 53 tokens
Split 7: 62 tokens
Split 8: 57 tokens
Split 9: 53 tokens
Split 10: 54 tokens
Split 11: 57 tokens
Split 12: 62 tokens
Split 13: 51 tokens
Split 14: 33 tokens
Split 15: 60 tokens
Split 16: 55 tokens
Split 17: 28 tokens
Split 18: 59 tokens
Split 19: 42 tokens
Split 20: 59 tokens
Split 21: 55 tokens
Split 22: 32 tokens
Split 23: 55 tokens
Split 24: 55 tokens
Split 25: 53 tokens
Split 26: 51 tokens
Split 27: 46 tokens
Split 28: 44 tokens
Split 29: 36 tokens
Split 30: 44 tokens
Split 31: 67 tokens
Split 32: 33 tokens
Split 33: 40 tokens
Split 34: 50 tokens
Split 35: 37 tokens
Split 36: 42 tokens
Split 37: 49 tokens
Split 38: 31 tokens
Split 39: 60 tokens
Split 40: 58 tokens
Split 41: 62 tokens
Split 42: 41 tokens
Split 43: 33 tokens
Split 44: 58 tokens
Split 45: 58 tokens
Split 46: 64 tokens
Split 47: 59 tokens
Split 48: 37 tokens
Split 49: 7 tokens
Split 50: 2

In [40]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from tqdm import tqdm

embeddings = OpenAIEmbeddings(
    api_key= GPT_API_KEY,
    base_url="https://api.gilas.io/v1/",
    model="text-embedding-3-large")
# 2. ساخت Vector Store

vector_store = InMemoryVectorStore(embeddings)

document_ids = []
for split in tqdm(split_docs, desc="Processing documents"):
    ids = vector_store.add_documents(documents=[split])
    document_ids.extend(ids)



Processing documents:   0%|          | 0/965 [00:00<?, ?it/s]

Processing documents: 100%|██████████| 965/965 [16:50<00:00,  1.05s/it]


In [41]:
query = "Where was time management discussed?"

results = vector_store.similarity_search(query, k=5)

for i, doc in enumerate(results):
    print(f"\n--- نتیجه {i+1} ---")
    print(" timesteps:", doc.metadata.get("start_time"), "تا", doc.metadata.get("end_time"))
    print("text:", doc.page_content)



--- نتیجه 1 ---
 timesteps: 01:17:31.760 تا 01:17:39.600
text: حالا تو ذهنم بود بعد نشستم یه منمایی نوشتم برای شرکت توی آخر اول نوامبر ۲۸

--- نتیجه 2 ---
 timesteps: 01:08:58.480 تا 01:09:06.359
text: اینستاگرام یکی بخواد می‌تونه ها روی این بحث یاد گرفتن از این مدلا و هم‌فکر داشتن

--- نتیجه 3 ---
 timesteps: 00:20:21.200 تا 00:20:24.559
text: راجع به کمپانی صحبت می‌کنی راجع به ایده صحبت می‌کنی راجع

--- نتیجه 4 ---
 timesteps: 01:26:47.320 تا 01:26:54.040
text: کنار این کالجه یه دونه جیم ما داشتیم می‌رفتیمش مثلاً بگو ساعت یک شبه جیمم

--- نتیجه 5 ---
 timesteps: 01:24:21.840 تا 01:24:26.600
text: ایده داری چون خیلی مثلا ما بحث حالا مثلا یه ایده خوب داشتیم مارکتینگر اشتباه
